### Draft Kit for Thugz Fantasy Hockey 2022-23

- Data: [Hashtag Hockey](https://hashtaghockey.com/fantasy-hockey-projections)
- Methodology: [VORP](https://en.wikipedia.org/wiki/Value_over_replacement_player)

Powered by 🟥 [redframes](https://github.com/maxhumber/redframes) 🕶

In [1]:
import redframes as rf

In [2]:
rf.__version__

'1.1'

### League Settings

In [3]:
TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}
GOALIE_CATEGORIES = ["wins", "saves", "save_percentage", "shutouts"]
SKATER_CATEGORIES = ["goals", "assists", "points", "powerplay_points", "shots_on_goal", "hits", "blocks"]

### Raw Projection Data

Scraped from [HashtagHockey](https://hashtaghockey.com/fantasy-hockey-projections)

In [4]:
raw = rf.load("data/projections.csv")
players = raw["PLAYER"]

In [5]:
raw.take(5)

,ADP,PLAYER,POS,TEAM,GP,TOI,G,A,P,+/-,SOG,HIT,BLK,PPP,GWG,SHO,W,SV%,GAA,SV,SA,TOTAL
0,1.1,Connor McDavid,C,EDM,79,22.1,0.53,0.99,1.52,0.34,3.83,0.90,0.33,0.55,0.07,NaN,NaN,(/),NaN,NaN,NaN,17.66
1,2.3,Leon Draisaitl,"C,LW",EDM,81,22.2,0.57,0.75,1.32,0.30,3.21,0.53,0.25,0.44,0.08,NaN,NaN,(/),NaN,NaN,NaN,13.66
2,3.1,Auston Matthews,C,TOR,77,20.5,0.82,0.55,1.36,0.26,4.64,0.92,0.74,0.44,0.07,NaN,NaN,(/),NaN,NaN,NaN,16.97
3,4.5,Nathan MacKinnon,C,COL,72,21.0,0.53,0.82,1.35,0.25,3.97,0.73,0.40,0.40,0.06,NaN,NaN,(/),NaN,NaN,NaN,13.21
4,5.5,Cale Makar,D,COL,74,24.1,0.24,0.73,0.97,0.41,2.36,1.00,0.94,0.35,0.05,NaN,NaN,(/),NaN,NaN,NaN,8.45


In [6]:
players[:5]

['Connor McDavid',
 'Leon Draisaitl',
 'Auston Matthews',
 'Nathan MacKinnon',
 'Cale Makar']

### Clean

In [7]:
from pandas import to_numeric

df = (
    raw.rename({
        'ADP': "adp",
        'PLAYER': "name",
        'POS': "position",
        'TEAM': "team",
        "GP": "games",
        'G': "goals",
        'A': "assists",
        'P': "points",
        'SOG': "shots_on_goal",
        'HIT': "hits",
        'BLK': "blocks",
        'PPP': "powerplay_points",
        'SHO': "shutouts",
        'W': "wins",
        'SV%': "save_percentage",
        'SV': "saves",
    })
    .split("position", into=["position", "secondary"], sep=",")
    .replace({"save_percentage": {"(/)": None}})
    .split("save_percentage", into=["save_percentage", "junk"], sep="(")
    .mutate({"save_percentage": lambda row: to_numeric(row["save_percentage"], errors="coerce")})
    .select(
        ['adp', 'name', 'position', "secondary", 'team', "games"] +
        SKATER_CATEGORIES +
        GOALIE_CATEGORIES
    )
)

In [8]:
df.sample(5, seed=42)

,adp,name,position,secondary,team,games,goals,assists,points,powerplay_points,shots_on_goal,hits,blocks,wins,saves,save_percentage,shutouts
0,169.0,Seth Jarvis,LW,RW,CAR,78,0.23,0.41,0.64,0.08,2.10,0.92,0.27,NaN,NaN,NaN,NaN
1,NaN,Spencer Martin,G,None,VAN,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.53,27.33,0.911,0.00
2,155.6,Elvis Merzlikins,G,None,CBJ,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.52,28.20,0.910,0.03
3,9.4,Nikita Kucherov,RW,None,TBL,77,0.53,0.90,1.43,0.61,3.41,0.46,0.20,NaN,NaN,NaN,NaN
4,183.9,Alex Nedeljkovic,G,None,DET,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.46,26.33,0.908,0.06


### Category Rollups (Means + STDs)

In [9]:
rollups = (
    df
    .filter(lambda row: row["games"] > 41)
    .select(["name"] + SKATER_CATEGORIES + GOALIE_CATEGORIES)
    .gather(SKATER_CATEGORIES + GOALIE_CATEGORIES, into=("category", "per_game"))
    .group("category")
    .rollup({
        "mean": ("per_game", rf.stat.mean),
        "std": ("per_game", rf.stat.std)
    })
)

In [10]:
rollups

,category,mean,std
0,assists,0.479013,0.147649
1,blocks,0.694292,0.526102
2,goals,0.283090,0.132759
3,hits,1.044206,0.690469
4,points,0.762876,0.232692
5,powerplay_points,0.186524,0.118225
6,save_percentage,0.913000,0.004406
7,saves,26.656400,1.322900
8,shots_on_goal,2.358155,0.580915
9,shutouts,0.052800,0.022083


### Category Z-Scores

In [11]:
zscores = df
for category in SKATER_CATEGORIES + GOALIE_CATEGORIES:
    row = rollups.filter(lambda row: row["category"] == category)
    mean = row["mean"][0]
    std = row["std"][0]
    zscores = zscores.mutate({category: lambda row: (row[category] - mean) / std})

In [12]:
zscores.take(5)

,adp,name,position,secondary,team,games,goals,assists,points,powerplay_points,shots_on_goal,hits,blocks,wins,saves,save_percentage,shutouts
0,1.1,Connor McDavid,C,None,EDM,79,1.859838,3.460817,3.253762,3.074453,2.533670,-0.208852,-0.692436,NaN,NaN,NaN,NaN
1,2.3,Leon Draisaitl,C,LW,EDM,81,2.161137,1.835343,2.394257,2.144022,1.466387,-0.744720,-0.844498,NaN,NaN,NaN,NaN
2,3.1,Auston Matthews,C,None,TOR,77,4.044251,0.480782,2.566158,2.144022,3.928023,-0.179887,0.086881,NaN,NaN,NaN,NaN
3,4.5,Nathan MacKinnon,C,None,COL,72,1.859838,2.309440,2.523183,1.805683,2.774669,-0.455062,-0.559382,NaN,NaN,NaN,NaN
4,5.5,Cale Makar,D,None,COL,74,-0.324575,1.699887,0.890123,1.382760,0.003177,-0.064023,0.467035,NaN,NaN,NaN,NaN


### "Best" Categories for each player

In [13]:
ranked_categories = (
    zscores
    .select(["name"] + SKATER_CATEGORIES + GOALIE_CATEGORIES)
    .gather(SKATER_CATEGORIES + GOALIE_CATEGORIES, into=("category", "z|score"))
    .replace({
        "category": {
            "goals": 'G',
            "assists": 'A',
            "points": 'P',
            "shots_on_goal": 'SOG',
            "hits": 'HIT',
            "blocks": 'BLK',
            "powerplay_points": 'PPP',
            "shutouts": 'SHO',
            "wins": 'W',
            "save_percentage": 'SV%',
            "saves": 'SV'
        }
    })
    .group("name")
    .rank("z|score", into="rank", descending=True)
    .drop("z|score")
    .sort(["name", "rank"])
    .group("name")
    .rollup({
        "categories": ("category", lambda x: x.str.cat(sep='|'))
    })
)

In [14]:
ranked_categories.take(5)

,name,categories
0,Aaron Ekblad,PPP|BLK|SOG|A|P|HIT|G
1,Adam Fox,BLK|A|PPP|P|HIT|G|SOG
2,Adam Henrique,G|BLK|SOG|PPP|HIT|P|A
3,Adin Hill,SV|SHO|SV%|W
4,Adrian Kempe,G|SOG|HIT|PPP|P|BLK|A


### Rolled-up Total Z-Score

In [15]:
ztotals = (
    zscores
    .fill("secondary", constant=0)
    .gather(GOALIE_CATEGORIES + SKATER_CATEGORIES, into=("category", "z|per_game"))
    .group(['name', "position", "secondary"])
    .rollup({"z|total": ("z|per_game", rf.stat.sum)})
    .sort("z|total", descending=True)
)

In [16]:
ztotals.take(5)

,name,position,secondary,z|total
0,Connor McDavid,C,0,13.281252
1,Auston Matthews,C,0,13.070230
2,Nikita Kucherov,RW,0,11.185083
3,Kirill Kaprizov,LW,0,10.696814
4,Nathan MacKinnon,C,0,10.258369


### Replacement Players

In [17]:
replacements = rf.DataFrame()
for position, count in ROSTER.items():
    value = (
        ztotals
        .filter(lambda row:
            (row["position"] == position) |
            (row["secondary"] == position)
        )
        .sort("z|total", descending=True)
        .take(count * TEAMS)
        .summarize({"replacement": ("z|total", rf.stat.mean)})
        ["replacement"][0]
    )
    row = rf.DataFrame({"position": [position], "replacement": [value]})
    replacements = replacements.append(row)

In [18]:
replacements

,position,replacement
0,C,5.185477
1,LW,5.406082
2,RW,3.702632
3,D,0.335624
4,G,1.204779


### Value Over Replacement PlayerS

In [19]:
vorps = (
    ztotals
    .join(replacements, on="position")
    .mutate({"vorp": lambda row: row["z|total"] - row["replacement"]})
    .sort("vorp", descending=True)
    .select(["name", "vorp"])
)

In [20]:
vorps.take(5)

,name,vorp
0,Connor McDavid,8.095774
1,Auston Matthews,7.884753
2,Nikita Kucherov,7.482451
3,Roman Josi,6.669942
4,Kirill Kaprizov,5.290732


### Draft Sheet

In [21]:
draft = (
    df
    .select(['team', 'name', 'position', 'secondary', "adp", "games"])
    .join(vorps, on="name")
    .join(ranked_categories, on="name")
    .sort("adp")
)

In [22]:
draft.take(10)

,team,name,position,secondary,adp,games,vorp,categories
0,EDM,Connor McDavid,C,None,1.1,79,8.095774,A|P|PPP|SOG|G|HIT|BLK
1,EDM,Leon Draisaitl,C,LW,2.3,81,3.226450,P|G|PPP|A|SOG|HIT|BLK
2,TOR,Auston Matthews,C,None,3.1,77,7.884753,G|SOG|P|PPP|A|BLK|HIT
3,COL,Nathan MacKinnon,C,None,4.5,72,5.072892,SOG|P|A|G|PPP|HIT|BLK
4,COL,Cale Makar,D,None,5.5,74,3.718760,A|PPP|P|BLK|SOG|HIT|G
5,MIN,Kirill Kaprizov,LW,None,7.0,80,5.290732,G|SOG|PPP|P|A|HIT|BLK
6,COL,Mikko Rantanen,C,RW,8.0,75,0.638550,P|A|PPP|G|SOG|BLK|HIT
7,TBL,Andrei Vasilevskiy,G,None,8.7,60,0.675140,W|SV%|SV|SHO
8,NYR,Igor Shesterkin,G,None,8.9,55,1.753977,W|SV%|SHO|SV
9,TBL,Nikita Kucherov,RW,None,9.4,77,7.482451,PPP|P|A|G|SOG|HIT|BLK
